In [6]:
english_to_french = [

    ("I am cold", "J'ai froid"),

    ("You are tired", "Tu es fatigué"),

    ("He is hungry", "Il a faim"),

    ("She is happy", "Elle est heureuse"),

    ("We are friends", "Nous sommes amis"),

    ("They are students", "Ils sont étudiants"),

    ("The cat is sleeping", "Le chat dort"),

    ("The sun is shining", "Le soleil brille"),

    ("We love music", "Nous aimons la musique"),

    ("She speaks French fluently", "Elle parle français couramment"),

    ("He enjoys reading books", "Il aime lire des livres"),

    ("They play soccer every weekend", "Ils jouent au football chaque week-end"),

    ("The movie starts at 7 PM", "Le film commence à 19 heures"),

    ("She wears a red dress", "Elle porte une robe rouge"),

    ("We cook dinner together", "Nous cuisinons le dîner ensemble"),

    ("He drives a blue car", "Il conduit une voiture bleue"),

    ("They visit museums often", "Ils visitent souvent des musées"),

    ("The restaurant serves delicious food", "Le restaurant sert une délicieuse cuisine"),

    ("She studies mathematics at university", "Elle étudie les mathématiques à l'université"),

    ("We watch movies on Fridays", "Nous regardons des films le vendredi"),

    ("He listens to music while jogging", "Il écoute de la musique en faisant du jogging"),

    ("They travel around the world", "Ils voyagent autour du monde"),

    ("The book is on the table", "Le livre est sur la table"),

    ("She dances gracefully", "Elle danse avec grâce"),

    ("We celebrate birthdays with cake", "Nous célébrons les anniversaires avec un gâteau"),

    ("He works hard every day", "Il travaille dur tous les jours"),

    ("They speak different languages", "Ils parlent différentes langues"),

    ("The flowers bloom in spring", "Les fleurs fleurissent au printemps"),

    ("She writes poetry in her free time", "Elle écrit de la poésie pendant son temps libre"),

    ("We learn something new every day", "Nous apprenons quelque chose de nouveau chaque jour"),

    ("The dog barks loudly", "Le chien aboie bruyamment"),

    ("He sings beautifully", "Il chante magnifiquement"),

    ("They swim in the pool", "Ils nagent dans la piscine"),

    ("The birds chirp in the morning", "Les oiseaux gazouillent le matin"),

    ("She teaches English at school", "Elle enseigne l'anglais à l'école"),

    ("We eat breakfast together", "Nous prenons le petit déjeuner ensemble"),

    ("He paints landscapes", "Il peint des paysages"),

    ("They laugh at the joke", "Ils rient de la blague"),

    ("The clock ticks loudly", "L'horloge tic-tac bruyamment"),

    ("She runs in the park", "Elle court dans le parc"),

    ("We travel by train", "Nous voyageons en train"),

    ("He writes a letter", "Il écrit une lettre"),

    ("They read books at the library", "Ils lisent des livres à la bibliothèque"),

    ("The baby cries", "Le bébé pleure"),

    ("She studies hard for exams", "Elle étudie dur pour les examens"),

    ("We plant flowers in the garden", "Nous plantons des fleurs dans le jardin"),

    ("He fixes the car", "Il répare la voiture"),

    ("They drink coffee in the morning", "Ils boivent du café le matin"),

    ("The sun sets in the evening", "Le soleil se couche le soir"),

    ("She dances at the party", "Elle danse à la fête"),

    ("We play music at the concert", "Nous jouons de la musique au concert"),

    ("He cooks dinner for his family", "Il cuisine le dîner pour sa famille"),

    ("They study French grammar", "Ils étudient la grammaire française"),

    ("The rain falls gently", "La pluie tombe doucement"),

    ("She sings a song", "Elle chante une chanson"),

    ("We watch a movie together", "Nous regardons un film ensemble"),

    ("He sleeps deeply", "Il dort profondément"),

    ("They travel to Paris", "Ils voyagent à Paris"),

    ("The children play in the park", "Les enfants jouent dans le parc"),

    ("She walks along the beach", "Elle se promène le long de la plage"),

    ("We talk on the phone", "Nous parlons au téléphone"),

    ("He waits for the bus", "Il attend le bus"),

    ("They visit the Eiffel Tower", "Ils visitent la tour Eiffel"),

    ("The stars twinkle at night", "Les étoiles scintillent la nuit"),

    ("She dreams of flying", "Elle rêve de voler"),

    ("We work in the office", "Nous travaillons au bureau"),

    ("He studies history", "Il étudie l'histoire"),

    ("They listen to the radio", "Ils écoutent la radio"),

    ("The wind blows gently", "Le vent souffle doucement"),

    ("She swims in the ocean", "Elle nage dans l'océan"),

    ("We dance at the wedding", "Nous dansons au mariage"),

    ("He climbs the mountain", "Il gravit la montagne"),

    ("They hike in the forest", "Ils font de la randonnée dans la forêt"),

    ("The cat meows loudly", "Le chat miaule bruyamment"),

    ("She paints a picture", "Elle peint un tableau"),

    ("We build a sandcastle", "Nous construisons un château de sable"),

    ("He sings in the choir", "Il chante dans le chœur")

]

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import matplotlib.pyplot as plt

# Define Vocabulary Class
class Vocabulary:
    def __init__(self):
        self.word2index = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2}
        self.index2word = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>"}
        self.word_count = {}
        self.n_words = 3  # Count default tokens

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.word_count[word] = 1
            self.n_words += 1
        else:
            self.word_count[word] += 1

# Define Dataset Class
class EngFrDataset(Dataset):
    def __init__(self, pairs, eng_vocab, fr_vocab):
        self.pairs = pairs
        self.eng_vocab = eng_vocab
        self.fr_vocab = fr_vocab

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        eng, fr = self.pairs[idx]
        eng_indexed = [self.eng_vocab.word2index[word] for word in eng.split(' ')]
        fr_indexed = [self.fr_vocab.word2index[word] for word in fr.split(' ')]
        return torch.tensor([self.eng_vocab.word2index["<SOS>"]] + eng_indexed + [self.eng_vocab.word2index["<EOS>"]], dtype=torch.long), torch.tensor([self.fr_vocab.word2index["<SOS>"]] + fr_indexed + [self.fr_vocab.word2index["<EOS>"]], dtype=torch.long)

def collate_fn(batch):
    eng_batch, fr_batch = zip(*batch)
    eng_batch_padded = pad_sequence(eng_batch, padding_value=0)
    fr_batch_padded = pad_sequence(fr_batch, padding_value=0)
    return eng_batch_padded, fr_batch_padded

# Define the Encoder
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

# Define the Decoder
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = torch.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

# Training function
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=50):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    loss = 0
    correct = 0
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    for ei in range(input_length):
        encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)

    decoder_input = torch.tensor([[1]], device=input_tensor.device)  # SOS_token
    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()

        # The key fix: Ensure target tensor passed to criterion is properly shaped.
        # Since target_tensor[di] is a scalar, we need to ensure it's not unsqueezed.
        loss += criterion(decoder_output, target_tensor[di])

        if topi.squeeze().item() == target_tensor[di].item():
            correct += 1

        if decoder_input.item() == 2:  # EOS_token
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    accuracy = correct / target_length if target_length > 0 else 0
    return loss.item() / target_length, accuracy



# Evaluation function to translate a sentence
def evaluate(encoder, decoder, sentence, eng_vocab, fr_vocab, max_length=50):
    with torch.no_grad():
        input_tensor = [eng_vocab.word2index[word] for word in sentence.split(' ')]
        input_tensor = torch.tensor([eng_vocab.word2index["<SOS>"]] + input_tensor + [eng_vocab.word2index["<EOS>"]], dtype=torch.long).view(-1, 1)
        encoder_hidden = encoder.initHidden()

        for ei in range(input_tensor.size(0)):
            encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

        decoder_input = torch.tensor([[1]], device=input_tensor.device)  # SOS_token
        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == 2:  # EOS_token
                break
            else:
                decoded_words.append(fr_vocab.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return ' '.join(decoded_words)
    
def clean_translation(translation):
    # Remove the start-of-string <SOS> and end-of-string <EOS> tokens from the translation
    cleaned_translation = translation.replace("<SOS>", "").replace("<EOS>", "").strip()
    return cleaned_translation

# Main training and testing pipeline
if __name__ == "__main__":
    eng_vocab = Vocabulary()
    fr_vocab = Vocabulary()

    for eng, fr in english_to_french:
        eng_vocab.add_sentence(eng)
        fr_vocab.add_sentence(fr)

    dataset = EngFrDataset(english_to_french, eng_vocab, fr_vocab)
    dataloader = DataLoader(dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)

    encoder = EncoderRNN(eng_vocab.n_words, 256)
    decoder = DecoderRNN(256, fr_vocab.n_words)
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=0.01)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=0.01)
    criterion = nn.NLLLoss()

    for epoch in range(10):  # Train for more epochs for better results
        for eng, fr in dataloader:
            train(eng.squeeze(0), fr.squeeze(0), encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)

    # Test the model with an example sentence
    test_sentence = "I am cold"
    print(f'Original: {test_sentence}')
    translated_sentence = evaluate(encoder, decoder, test_sentence, eng_vocab, fr_vocab)
    print(f'Translated: {translated_sentence}')


Original: I am cold
Translated: <SOS> Elle étudie la
